In [276]:
import json
import re
import ast

from itertools import zip_longest

In [277]:
json_file_path = "seed_data_final_Nov18.json"

In [278]:
json_file = open(json_file_path, 'r')

In [279]:
data = json.load(json_file)

In [280]:
# if number then int, [] then list, some alphanumeric then char, if True/False then boolean

In [282]:
for datum in data:
    # MATCH INPUT AND OUTPUT
    match = re.search(r'\w+\((.*?)\)\s*==\s*(.*)', datum['asserts'][0])
    outputs = ast.literal_eval(match.group(2))

    # recursively apply string match to find actual input params
    # some hardcoding to ignore fucked up cases
    if match.group(1) != '"a)b(c)d"':
        inputs_match = None
        while match is not None:
            inputs_match = match
            match = re.search(r'\w+\((.*?)\)\s*', match.group(1))
        inputs = ast.literal_eval(inputs_match.group(1))
    else:
        inputs = ast.literal_eval(match.group(1))

    # zipping for later 
    if isinstance(inputs, tuple):
        inputs = list(inputs)
    else:
        inputs = [inputs]

    # MATCH FUNCTION NAME TO MAKE TYPED
    match = re.search(r'assert (.*?)\((.*?)\)\s*==\s*(.*)', datum['asserts'][0])
    func_name = match.group(1)
    # some hardcoding to ignore fucked up cases
    if func_name == "set":
        inner_match = re.search(r'(.*)\((.*?)\)\s*', match.group(2))
        func_name = inner_match.group(1)

    # find func_name and extract that line from code
    func_def_line = ""
    for line in datum['code'].splitlines():
        if "def " + func_name in line:
            func_def_line = line
            break
    
    # IF FUNC_DEF IS ALREADY TYPED, IGNORE
    if "->" not in func_def_line:
        print(func_def_line)
        match = re.search(r'(.*)\((.*?)\)\s*:\s*', func_def_line)
        # assumption: vars are separated by ", "
        var_names = match.group(2).split(", ")
        typed_signature = match.group(1) + '('
        first_var = True
        for var_name, input in zip_longest(var_names, inputs):
            # add comma only after first var
            if first_var == True:
                first_var = False
            else:
                typed_signature = typed_signature + ", "

            # if var already has type
            if ":" in var_name:
                typed_signature = typed_signature + var_name
            else:
                # if var has default value
                if "=" in var_name:
                    v_name = var_name.split("=")[0]
                    default_val = var_name.split("=")[1]
                    typed_signature = typed_signature + v_name + ": " + type(ast.literal_eval(default_val)).__name__ + " = " + default_val
                else:
                    typed_signature = typed_signature + var_name + ": " + type(input).__name__
        typed_signature = typed_signature + ') -> ' + type(outputs).__name__ + ':' 
        code = datum['code']
        new_code = ""
        for line in code.splitlines():
            if "def " + func_name in line:
                new_code = new_code + typed_signature + "\n"
            else:
                new_code = new_code + line + "\n"
        datum['code'] = new_code

In [287]:
processed_json_file_path = "processed_" + json_file_path 

In [288]:
with open(processed_json_file_path, 'w') as processed_file:
    json.dump(data, processed_file)

In [283]:
#debug

In [233]:
type(str(type("sa")))

str

In [240]:
"hello " + type(2).__name__

'hello int'

In [190]:
data[0]['code']

'def middlesum(mat):\n\n        row_sum = 0\n        col_sum = 0\n        n = len(mat)\n\n        for i in range(n):\n                row_sum += mat[n // 2][i]\n        \n        for i in range(n):\n                col_sum += mat[i][n // 2]\n\n        mid_sum = [row_sum, col_sum]\n        return mid_sum\n        \n        '

In [136]:
eval = ast.literal_eval("[1, 23, 12, 9, 30, 2, 50], 3")

In [140]:
type(eval[1])

int

In [172]:
string = "assert minRemoveToMakeValid(\"a)b(c)d\")==\"ab(c)d\""

In [173]:
match = re.search(r'\w+\((.*?)\)\s*==\s*(.*)', string)

In [180]:
match.group(1)!= '"a)b(c)d"'

False

In [175]:
match.group(2)

'"ab(c)d"'

In [177]:
ast.literal_eval(match.group(1))

'a)b(c)d'

In [166]:
match = re.search(r'\w+\((.*?)\)', match.group(1))

In [167]:
match.group(1)

'c'

In [193]:
data[0]['asserts'][0]

'assert middlesum([[2, 5, 7],[3, 7, 2],[5, 6, 9]]) == [12, 18]'

In [219]:
match = re.search(r'(.*)\((.*?)\):', "def maxDistinctNum(a, k):")

In [225]:
match.group(2).split(", ")

['a', 'k']

In [226]:
match.group(1)

'def maxDistinctNum'

In [227]:
a = [1,2,3]

In [256]:
b = [4,5]

In [257]:
for x,y in zip(a,b):
    print(x)
    print(y)

1
4
2
5


In [242]:
"x = 5".split("=")

['x ', ' 5']

In [261]:
list(([1,2,3],4))

[[1, 2, 3], 4]

In [262]:
list(4)

TypeError: 'int' object is not iterable

In [263]:
if (1,2) is iterable:
    print("hello")

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/v0/g401c4x979d___jdl5ddw2_r0000gn/T/ipykernel_18515/4053484565.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (1,2) is iterable:


NameError: name 'iterable' is not defined

In [265]:
is_iterable(4)

False

In [266]:
is_iterable((1,2,[1,2,3]))

True

In [270]:
isinstance((1,2), tuple)

True